In [95]:
# import os
# os.chdir("Desktop")

In [96]:
import pandas as pd
df = pd.read_csv("twitter_train.csv", encoding ="Latin")

In [97]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [98]:
col_names = ["Target","ID", "Date", "Flag", "User","Text"]

In [99]:
row = pd.DataFrame(list(df.columns)).T

In [100]:
row.columns = col_names

In [101]:
df.columns = col_names

In [102]:
df.head()

,Target,ID,Date,Flag,User,Text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [103]:
df= df.append(row)

In [104]:
#Transform the target variable using label encoding- we will keep it 0 and 1 for ease of interpretation

df["Target"]= df["Target"].apply(lambda x:1 if x==4 else 0)

In [105]:
df["Target"].value_counts()

1    800000
0    800000
Name: Target, dtype: int64

In [106]:
#this dataset is too huge for me to run on my local machine- I will use a sample

In [107]:
pos= df.loc[df["Target"]==1]

In [108]:
neg= df.loc[df["Target"]==0]

In [109]:
pos_sample= pos.sample(frac=0.1)

In [110]:
neg_sample= neg.sample(frac=0.1)

In [111]:
X= pd.concat([pos_sample, neg_sample])

In [112]:
X= X.sample(frac=1)

In [113]:
len(X)

160000

In [114]:
y= X["Target"]

In [115]:
len(y)

160000

In [116]:
X= X.drop(["Target"],1)

In [117]:
len(X)

160000

In [118]:
X.Flag.value_counts()

NO_QUERY    160000
Name: Flag, dtype: int64

In [119]:
X.drop(["ID","Flag", "User"],1,inplace=True)

In [120]:
#Working with dates

In [121]:
#Split into a list and get the first, third and fifth elements and then order them like 2019-05-09

In [122]:
X["Date"]= X["Date"].str.split(" ")

In [123]:
X["Year"]= X["Date"].str.get(-1)

In [124]:
X["Month"]= X["Date"].str.get(1)

In [125]:
#Drop the original Date column
X.drop(["Date"],1,inplace=True)

In [126]:
X.head()

,Text,Year,Month
1346587,@nicklivingston WTF?!?! this series might just...,2009,Jun
769248,@1045CHUMFM richie don't go silent on the GG's...,2009,Jun
44170,@queenlaytho this no car thing is hard and I'...,2009,May
625143,@eglamourphoto I cabt dm from my phne my palm...,2009,Jun
10598,Not going to the baseball game with dakota tod...,2009,Apr


In [127]:
text= X["Text"]

In [128]:
import re
import nltk
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [129]:
import numpy as np
normalize_corpus = np.vectorize(normalize_document)

In [130]:
norm_corpus = normalize_corpus(text)

In [131]:
#Lets first apply a Bag of Words model

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
# cv_matrix = cv.fit_transform(norm_corpus)
# cv_matrix = cv_matrix.toarray()
# cv_matrix

In [132]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test= tts(norm_corpus, y, test_size=0.3, random_state= 42)

In [133]:
X_train = cv.fit_transform(X_train)
X_test= cv.transform(X_test)

In [134]:
# get all unique words in the corpus
vocab = cv.get_feature_names()
# show document feature vectors
matrix = pd.DataFrame(X_train.toarray(), columns=vocab)

In [135]:
matrix.head()

,aa,aaa,aaaa,aaaaa,aaaaaa,aaaaaaa,aaaaaaaaa,aaaaaaaaaaaa,aaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaa,...,zzzzzzgoodnightt,zzzzzzs,zzzzzzz,zzzzzzzz,zzzzzzzzz,zzzzzzzzzz,zzzzzzzzzzzz,zzzzzzzzzzzzzz,zzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
len(matrix)

112000

In [137]:
from sklearn.naive_bayes import MultinomialNB
mnb= MultinomialNB()

In [138]:
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [139]:
y_pred_mnb= mnb.predict(X_test)

In [140]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

In [141]:
accuracy_score(y_test, y_pred_mnb)

0.7521875

In [142]:
confusion_matrix(y_test, y_pred_mnb)

array([[19440,  4549],
       [ 7346, 16665]])

In [143]:
roc_auc_score(y_test, y_pred_mnb)

0.752214155413043

In [144]:
#Let us now use a boosting algorithm to see if we can get higher accuracy
from xgboost import XGBClassifier

In [145]:
xgb= XGBClassifier()

In [146]:
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [147]:
y_pred_xgb= xgb.predict(X_test)

In [148]:
accuracy_score(y_test, y_pred_xgb)

0.6754583333333334

In [149]:
roc_auc_score(y_test, y_pred_xgb)

0.6753682599297908

In [150]:
confusion_matrix(y_test, y_pred_xgb)

array([[11487, 12502],
       [ 3076, 20935]])

In [ ]:
#It seems that the boosting algorithm does not give us higher accuracy than our Naive Bayes algorithm

In [49]:
#Now lets try to use Naive Bayes with a bag of N-grams model
bv = CountVectorizer(ngram_range=(2,2))

In [50]:
X_train, X_test, y_train, y_test= tts(norm_corpus, y, test_size=0.3, random_state= 42)

In [51]:
X_train = bv.fit_transform(X_train)

In [52]:
X_test= bv.transform(X_test)

In [53]:
# get all unique words in the corpus
vocab = bv.get_feature_names()
# show document feature vectors
matrix = pd.DataFrame(X_train.toarray(), columns=vocab)

In [54]:
matrix.head()

,aa amp,aa board,aa gai,aa history,aa jonas,aa long,aa man,aa mtg,aa ouch,aa rahi,...,zzzzy office,zzzzz night,zzzzzs working,zzzzzz inspired,zzzzzzs sweet,zzzzzzzgoodnight tweets,zzzzzzzzs got,zzzzzzzzzzzschool againltbleegt,zzzzzzzzzzzzzzzzzzzzzzzzzzz sweet,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz tired
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [56]:
y_pred_bgram= mnb.predict(X_test)

In [58]:
accuracy_score(y_test, y_pred_bgram)

0.6617916666666667

In [92]:
confusion_matrix(y_test, y_pred_bgram)

array([[18775,  5176],
       [11058, 12991]])

In [93]:
roc_auc_score(y_test, y_pred_bgram)

0.662040447151239

In [59]:
#Lets now try a tri-gram based model
tv = CountVectorizer(ngram_range=(3,3))

In [60]:
X_train, X_test, y_train, y_test= tts(norm_corpus, y, test_size=0.3, random_state= 42)

In [61]:
X_train = tv.fit_transform(X_train)

In [63]:
X_test=tv.transform(X_test)

In [64]:
# get all unique words in the corpus
vocab = tv.get_feature_names()
# show document feature vectors
matrix = pd.DataFrame(X_train.toarray(), columns=vocab)

In [65]:
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [66]:
y_pred_tgram= mnb.predict(X_test)

In [67]:
accuracy_score(y_test, y_pred_tgram)

0.5325208333333333

In [90]:
confusion_matrix(y_test, y_pred_tgram)

array([[22215,  1736],
       [20703,  3346]])

In [91]:
roc_auc_score(y_test, y_pred_tgram)

0.533325644123041

In [68]:
#Our accuracy is reducing with the number of grams we use and the original bag of words model is best

In [75]:
#lets now try a TFIDF model

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [77]:
tfidf= TfidfVectorizer()

In [78]:
X_train, X_test, y_train, y_test= tts(norm_corpus, y, test_size=0.3, random_state= 42)

In [79]:
X_train= tfidf.fit_transform(X_train)

In [80]:
X_test = tfidf.transform(X_test)

In [81]:
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [82]:
y_pred_tfidf= mnb.predict(X_test)

In [83]:
accuracy_score(y_test, y_pred_tfidf)

0.7486458333333333

In [88]:
confusion_matrix(y_test, y_pred_tfidf)

array([[19459,  4492],
       [ 7573, 16476]])

In [89]:
roc_auc_score(y_test, y_pred_tfidf)

0.7487758356089953

In [ ]:
#Try some boosting algorithms 

In [84]:
#Very slightly lower than the Bag of Words result

In [ ]:
#The final result will be using a Bag of Words model with Naive Bayes- giving us an accuracy of 75%